In [ ]:
!pip install --quiet torch
!pip install --quiet transformers[torch]
!pip install --quiet accelerate -U
!pip install --quiet sentencepiece
!pip install --quiet transformers

In [ ]:
!pip install --quiet transformers[torch] --upgrade
!pip install --quiet accelerate --upgrade

In [ ]:
import torch
import pandas as pd
from transformers import BertLMHeadModel , AutoTokenizer
from transformers import Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader

In [ ]:


# قراءة الملف CSV
data = pd.read_excel('/content/drive/MyDrive/marwah_projects/news35.xlsx')
shu_df=data.sample(frac=1).reset_index(drop=True)
#shu_df.to_csv('/content/drive/MyDrive/generation&marwah_dala/th2.csv', index=False)

#data = pd.read_csv('/content/drive/MyDrive/generation&marwah_dala/th2.csv')
# عدد الصفوف المطلوب استخراجها
num_rows = 1000  # يمكنك تعيين القيمة التي تريدها

# استخراج الجزء المطلوب من البيانات
subset = data.head(num_rows)

# القيام بالعمل المطلوب على الجزء المستخرج
# يمكنك إجراء العمليات الإضافية التي تحتاجها هنا

# حفظ الجزء المستخرج إلى ملف CSV جديد (اختياري)
subset.to_csv('/content/drive/MyDrive/marwah_projects/news35.csv', index=False)

In [ ]:
import pandas as pd

#data = pd.read_csv('/content/drive/MyDrive/multytask_marwah_news/data_cleaned2.csv')
data = pd.read_csv('/content/drive/MyDrive/marwah_projects/news35.csv')
print('Shape of the Dataframe: ', data.shape)
display(data.head())

Shape of the Dataframe:  (1000, 3)


,Title,Text,Category
0,\nسرقة 70 لوحة يقدر ثمنها بـ 600 ألف يورو,سرق ثلاثة لصوص 70 لوحة من معرض باب القلعة، إسب...,ثقافي
1,\nمجلة أمريكية : السينما المغربية من بين الأكث...,كتبت مجلة (فاريتي) الأمريكية المتخصصة في الشؤو...,ثقافي
2,\nوفاة الممثل المصري عمر الحريري,أخبارنا المغربية - متابعة\n\nتوفي مساء الاحد ب...,ثقافي
3,\nمن المحلية إلى العالمية .. الزربية البجعدية ...,أرشيفية\n\n \nتجسد الزربية البجعدية، التي تتمي...,ثقافي
4,"\n""سراب"" في مراكش .. فيلم يذكي الحنين إلى العص...","أرشيفية\n\n \nيمنح فيلم ""سراب"" للمخرج المجري ز...",ثقافي


In [ ]:
print(data.columns)

Index(['Title', 'Text', 'Category'], dtype='object')


In [ ]:
len(data)

1000

In [ ]:
!pip install --quiet klib sentencepiece transformers
import klib

In [ ]:
# Data cleaning
dataframe=klib.data_cleaning(data)
dataframe=data.drop_duplicates(subset='Text', keep=False)
dataframe=data.drop_duplicates(subset='Category', keep=False)
dataframe=data.drop_duplicates(subset='Title', keep=False)

Shape of cleaned data: (1000, 2) - Remaining NAs: 1


Dropped rows: 0
     of which 0 duplicates. (Rows (first 150 shown): [])

Dropped columns: 1
     of which 1 single valued.     Columns: ['category']
Dropped missing values: 0
Reduced memory by at least: 0.0 MB (-0.0%)



In [ ]:
import torch
import torch.nn as nn
from transformers import Trainer, TrainingArguments

from tqdm.auto import tqdm
import pandas as pd
import numpy as np

In [ ]:
dataframe['Text'] = '<idf.news> ' + dataframe['Text']
dataframe['Category'] = '<tupe.new> ' + dataframe['Category']
display(dataframe)

,Title,Text,Category
0,\nسرقة 70 لوحة يقدر ثمنها بـ 600 ألف يورو,<idf.news> سرق ثلاثة لصوص 70 لوحة من معرض باب ...,<tupe.new> ثقافي
1,\nمجلة أمريكية : السينما المغربية من بين الأكث...,<idf.news> كتبت مجلة (فاريتي) الأمريكية المتخص...,<tupe.new> ثقافي
2,\nوفاة الممثل المصري عمر الحريري,<idf.news> أخبارنا المغربية - متابعة\n\nتوفي م...,<tupe.new> ثقافي
3,\nمن المحلية إلى العالمية .. الزربية البجعدية ...,<idf.news> أرشيفية\n\n \nتجسد الزربية البجعدية...,<tupe.new> ثقافي
4,"\n""سراب"" في مراكش .. فيلم يذكي الحنين إلى العص...","<idf.news> أرشيفية\n\n \nيمنح فيلم ""سراب"" للمخ...",<tupe.new> ثقافي
...,...,...,...
995,\nالملحن المصري حلمي بكر يوجه كلاما قاسيا لسعد...,<idf.news> أخبارنا المغربية : سناء الوردي\nفي ...,<tupe.new> ثقافي
996,"\nنادية المنصوري لـ "" اخبارنا "" : الإمارات احت...",<idf.news> عبدالاله بوسحابة : اخبارنا المغربية...,<tupe.new> ثقافي
997,\nالمغني الشعبي عبد الله الداودي يتسبب في سجن ...,<idf.news> المغني الشعبي عبد الله الداودي يتسب...,<tupe.new> ثقافي
998,\nحقيقة زواج الفنان كاظم الساهر من فتاة مغربية !,<idf.news> حقيقة زواج الفنان كاظم الساهر من فت...,<tupe.new> ثقافي


In [ ]:

def form_string(text, category):
    text_str = str(text)  # Convert to string
    category_str = str(category)  # Convert to string
    s = f"<|startoftext|>\n{text_str.strip()}\n\n{category_str.strip()}"
    return s

In [ ]:
target = dataframe.apply(lambda x:form_string(x['Text'],x['Category']),axis=1).to_list()
input=dataframe['Title']

In [ ]:
model_name='sultan/ArabicT5-Base'

In [ ]:
model_save_path = '/content/drive/MyDrive/marwah_projects'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name,
                                              bos_token='<|startoftext|>',
                                              eos_token='<|endoftext|>',
                                              unk_token='<|unknown|>',
                                              pad_token='<|pad|>'
                                             )
new_special_tokens = ['<idf.news>', '<tupe.news>']
model = BertLMHeadModel.from_pretrained(model_name)
model.resize_token_embeddings(len(tokenizer))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
You are using a model of type t5 to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
Some weights of BertLMHeadModel were not initialized from the model checkpoint at sultan/ArabicT5-Base and are newly initialized: ['encoder.layer.9.output.LayerNorm.bias', 'encoder.layer.4.attention.output.LayerNorm.bias', 'encoder.layer.1.output.dense.bias', 'encoder.layer.2.attention.self.query.weight', 'encoder.layer.10.intermediate.dense.bias', 'encoder.layer.6.output.dense.weight', 'encoder.layer.0.attention.self.value.weight', 'encoder.layer.5.intermediate.dense.weight', 'encoder.layer.8.attention.self.key.bias', 'encoder.layer.

Embedding(32104, 768)

In [ ]:
len(tokenizer.vocab)

32104

In [ ]:
print(model.config.max_length)

20


In [ ]:
LANG_TOKEN_MAPPING = {
    'identify language': '<idf.news>',
    'news tupe': '<tupe.news>'
}
special_tokens_dict = {'additional_special_tokens': list(LANG_TOKEN_MAPPING.values())}
special_tokens_dict

{'additional_special_tokens': ['<idf.news>', '<tupe.news>']}

In [ ]:
tokenizer.add_special_tokens(special_tokens_dict)

2

In [ ]:
len(tokenizer)

32106

In [ ]:
tokenizer.all_special_tokens

['<|startoftext|>',
 '<|endoftext|>',
 '<|unknown|>',
 '<|pad|>',
 '<idf.news>',
 '<tupe.news>']

In [ ]:
model.config.vocab_size

32104

In [ ]:
model.resize_token_embeddings(len(tokenizer))

You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embedding dimension will be 32106. This might induce some performance reduction as *Tensor Cores* will not be available. For more details about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc


Embedding(32106, 768)

In [ ]:
model.config.vocab_size

32106

In [ ]:
tokenizer.save_pretrained(model_save_path)

('/content/drive/MyDrive/marwah_projects/tokenizer_config.json',
 '/content/drive/MyDrive/marwah_projects/special_tokens_map.json',
 '/content/drive/MyDrive/marwah_projects/spiece.model',
 '/content/drive/MyDrive/marwah_projects/added_tokens.json',
 '/content/drive/MyDrive/marwah_projects/tokenizer.json')

In [ ]:
tokenizer.convert_tokens_to_ids(['<|pad|>'])

[32103]

In [ ]:
tokenizer.convert_tokens_to_ids(['<|startoftext|>'])

[32100]

In [ ]:
tokenizer.convert_tokens_to_ids(['<|endoftext|>'])

[32101]

In [ ]:
tokenizer.convert_tokens_to_ids(['<|unknown|>'])

[32102]

In [ ]:
tokenizer.convert_tokens_to_ids(['<|<idf.news>|>'])

[32102]

In [ ]:
tokenizer.convert_tokens_to_ids(['<|<tupe.new>|>'])

[32102]

In [ ]:
def generate(prompt):
    inputs = tokenizer.encode_plus(prompt, return_tensors='pt')
    output = model.generate(**inputs,max_length=256,do_sample=True,pad_token_id=32103)
    print(tokenizer.decode(output[0]))

In [ ]:
tokenizer.special_tokens_map

{'bos_token': '<|startoftext|>',
 'eos_token': '<|endoftext|>',
 'unk_token': '<|unknown|>',
 'pad_token': '<|pad|>',
 'additional_special_tokens': ['<idf.news>', '<tupe.news>']}

In [ ]:
num_special_tokens = len(tokenizer.all_special_tokens)
print(num_special_tokens)

6


In [ ]:
len(tokenizer)

32106

In [ ]:
from transformers import AutoTokenizer

model_name = 'sultan/ArabicT5-Base'
tokenizer = AutoTokenizer.from_pretrained(model_name)

vocab = tokenizer.get_vocab()
all_tokens = list(vocab.keys())

print(all_tokens)

['ائ', '▁أشبه', '▁2000،', '▁الشلهوب', '▁جوازات', 'شارب', '06', 'عترف', '▁طرف', '▁فلا', '▁زب', '▁سلوك', '▁والصرف', 'وه', 'وافد', '▁بحسن', '▁والاداري', '▁لمونديال', 'جس', '▁المولد', 'لحن', '▁تعذيب', 'تأزيم', '▁شنودة', '▁وهجر', '▁الكرتون', '▁بسيارته', '▁والأمهات', '▁يكمن', '▁بخار', '▁الناصر', '▁الطمأنينة', '▁الآتي', '▁لتشكل', '▁للعيش', '▁هجمات', '▁الخارطة', 'شتري', '▁والر', '▁صدرت', 'شاغل', '▁تدعو', '▁فريدريك', '▁يعالج', '▁تكتيك', '▁تفكيك', '▁طقس', '▁محذرا', '▁لاقتصاد', '▁والكويت', 'اشي', '▁الاستطلاع', '▁مسؤولين', '▁أعرب', '▁أيام', '▁بمعلومات', '▁الطبقة', 'ويعرف', '▁1,3', 'سلح', '▁للمدرب', 'ob', '▁عصابة', '▁بالقليوبية', '▁سواء', '▁تبذلها', '▁يمكنها', '▁الدستورى', '<extra_id_83>', 'رج', '▁النجاة', '▁ناقد', '▁ظهير', '▁الحكام', '▁الليبية', 'خار', '▁الحزين', '▁عشيرة', '▁الصلبة', '▁وكانوا', 'إسمنت', '▁والمعلومات', '▁وقام', '▁تسبب', '▁حديقة', '▁والتقى', '▁اكتشفت', '▁تحديات', '▁عصره', '▁ريف', '▁أنت', '▁التشكيلي', '▁فوجئت', '▁وعرف', '▁تهدم', '▁بالقطيف', '▁ادراج', '▁تهدد', '▁عابرة', '▁عددا', '▁وحي

In [ ]:
from transformers import AutoTokenizer

model_name = 'sultan/ArabicT5-Base'
tokenizer = AutoTokenizer.from_pretrained(model_name)

vocab_size = len(tokenizer.get_vocab())

print("عدد الرموز الكلي (all_tokens):", vocab_size)

عدد الرموز الكلي (all_tokens): 32100


In [ ]:
tokenizer.all_special_tokens

['</s>',
 '<unk>',
 '<pad>',
 '<extra_id_0>',
 '<extra_id_1>',
 '<extra_id_2>',
 '<extra_id_3>',
 '<extra_id_4>',
 '<extra_id_5>',
 '<extra_id_6>',
 '<extra_id_7>',
 '<extra_id_8>',
 '<extra_id_9>',
 '<extra_id_10>',
 '<extra_id_11>',
 '<extra_id_12>',
 '<extra_id_13>',
 '<extra_id_14>',
 '<extra_id_15>',
 '<extra_id_16>',
 '<extra_id_17>',
 '<extra_id_18>',
 '<extra_id_19>',
 '<extra_id_20>',
 '<extra_id_21>',
 '<extra_id_22>',
 '<extra_id_23>',
 '<extra_id_24>',
 '<extra_id_25>',
 '<extra_id_26>',
 '<extra_id_27>',
 '<extra_id_28>',
 '<extra_id_29>',
 '<extra_id_30>',
 '<extra_id_31>',
 '<extra_id_32>',
 '<extra_id_33>',
 '<extra_id_34>',
 '<extra_id_35>',
 '<extra_id_36>',
 '<extra_id_37>',
 '<extra_id_38>',
 '<extra_id_39>',
 '<extra_id_40>',
 '<extra_id_41>',
 '<extra_id_42>',
 '<extra_id_43>',
 '<extra_id_44>',
 '<extra_id_45>',
 '<extra_id_46>',
 '<extra_id_47>',
 '<extra_id_48>',
 '<extra_id_49>',
 '<extra_id_50>',
 '<extra_id_51>',
 '<extra_id_52>',
 '<extra_id_53>',
 '<extra_

In [ ]:
model.config.vocab_size

32106

In [ ]:
def print_recipe(idx):
    print(f"{data['Title'][idx]}\n\n{data['Text'][idx]}\n\n{data['Category'][idx]}")

In [ ]:
#def form_string(title,content,category):
    #s = f"<|startoftext|>Title:\n{title.strip()}\n\nContent:\n{content.strip()}\n\ncategory:\n{category.strip()}<|endoftext|>"
    #s = f"<|startoftext|>Title:\n{title.strip()}\n\nContent:\n{content.strip()}<|endoftext|>"
    #return s

In [ ]:
#def form_string(category,content):
    #s = f"<|startoftext|>Title:\n{title.strip()}\n\nContent:\n{content.strip()}<|endoftext|>"
    #s = f"<|startoftext|>idf.news:\n{category.strip()}\n\ntupe.news:\n{content.strip()}<|endoftext|>"
    #return s

In [ ]:
#data2 = data.apply(lambda x:form_string(x['titel_news'],x['Content'],x['Category']),axis=1).to_list()

In [ ]:
data2 = data.apply(lambda x:form_string(x['Category'],x['Text']),axis=1).to_list()

In [ ]:
train_size = 0.80
train_len = int(train_size * len(data2))
train_data = data2[:train_len]
val_data = data2[train_len:]

In [ ]:
class RecipeDataset:
    def __init__(self,data):
        self.data = data
        self.input_ids = []
        self.attn_masks = []

        for item in tqdm(data):
            encodings = tokenizer.encode_plus(item,
                                              truncation=True,
                                              padding='max_length',
                                              max_length=512,
                                              return_tensors='pt'
                                             )
            self.input_ids.append(torch.squeeze(encodings['input_ids'],0))
            self.attn_masks.append(torch.squeeze(encodings['attention_mask'],0))

    def __len__(self):
        return len(self.data)

    def __getitem__(self,idx):
        return self.input_ids[idx], self.attn_masks[idx]

In [ ]:
def collate_fn(batch):
    return {
        'input_ids': torch.stack([item[0] for item in batch]),
        'attention_mask': torch.stack([item[1] for item in batch]),
        'labels': torch.stack([item[0] for item in batch])
    }

In [ ]:
#train_ds = RecipeDataset(train_data)
#val_ds = RecipeDataset(val_data)

In [ ]:
train_ds = RecipeDataset(input)
val_ds = RecipeDataset(target)

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

In [ ]:
args = TrainingArguments(output_dir=model_save_path,
                         per_device_train_batch_size=2,
                         per_device_eval_batch_size=2,
                         gradient_accumulation_steps=2,
                         report_to='none',
                         num_train_epochs=3,
                         save_strategy='no'
                        )

In [ ]:
optim = torch.optim.AdamW(model.parameters(),lr=5e-5)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optim,20,eta_min=1e-7)

In [ ]:
trainer = Trainer(model,
                  args,
                  train_dataset=train_ds,
                  eval_dataset=val_ds,
                  data_collator=collate_fn,
                  optimizers=(optim,scheduler)
                 )

In [ ]:
trainer.train()

Step,Training Loss
500,0.194800


TrainOutput(global_step=750, training_loss=0.17666250101725262, metrics={'train_runtime': 412.0506, 'train_samples_per_second': 7.281, 'train_steps_per_second': 1.82, 'total_flos': 1200236135546880.0, 'train_loss': 0.17666250101725262, 'epoch': 3.0})

In [ ]:
trainer.save_model()

In [ ]:
import torch
from transformers import T5ForConditionalGeneration, AutoTokenizer

# تحديد المسارات للموديل والرمزنة
model_path = '/content/drive/MyDrive/marwah_projects'
tokenizer_path = '/content/drive/MyDrive/marwah_projects'

# استيراد الموديل
model = T5ForConditionalGeneration.from_pretrained(model_path)

# استيراد الرمزنة
tokenizer =  AutoTokenizer.from_pretrained(tokenizer_path)

You are using a model of type bert to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.
Some weights of T5ForConditionalGeneration were not initialized from the model checkpoint at /content/drive/MyDrive/marwah_projects and are newly initialized: ['decoder.block.12.layer.1.EncDecAttention.q.weight', 'decoder.block.13.layer.1.EncDecAttention.q.weight', 'encoder.block.2.layer.0.SelfAttention.v.weight', 'encoder.block.9.layer.0.layer_norm.weight', 'decoder.block.16.layer.1.EncDecAttention.k.weight', 'decoder.block.5.layer.1.EncDecAttention.k.weight', 'decoder.block.3.layer.0.layer_norm.weight', 'encoder.block.9.layer.0.SelfAttention.o.weight', 'decoder.block.6.layer.2.layer_norm.weight', 'decoder.block.12.layer.2.DenseReluDense.wi.weight', 'encoder.block.1.layer.1.DenseReluDense.wo.weight', 'decoder.block.2.layer.0.SelfAttention.k.weight', 'encoder.block.7.layer.0.SelfAttention.q.weight', 'decoder.block.5.layer.1.layer_norm.weight

In [ ]:
def create_prompt(ingredients):
    s = f"<|startoftext|>idf.news:\n{ingredients}\n\nText:\n"
    return s

In [ ]:
ingredients = ['دشّنت أكاديمية العلوم الشرطية بالشارقة موقعها الإلكتروني الجديد خلال فعاليات معرض جيتكس 2017 بدبي']
#ingredients = ['أعلنت شركة هيرست كورب . سكيف الخاصة بصناعة أجهزة القارئ الإلكتروني مؤخراً أنها وقعت اتفاقية مع إدارة الهواتف النقالة بشركة سامسونج لتصبح شريك خدمات القراءة الإلكترونية المفضل لشركة سامسونج .يعني هذا أن الهواتف الذكية المستقبلية لشركة سامسونج']

In [ ]:
def generate(prompt):
    inputs = tokenizer.encode_plus(prompt, return_tensors='pt')
    output = model.generate(inputs['input_ids'],max_length=256,do_sample=True,pad_token_id=32103)
    print(tokenizer.decode(output[0]))

In [ ]:
model.eval()
input_text = create_prompt(ingredients)
generate(input_text)

<pad> أبو ميلان كلام تدني للمؤسسة للمؤسسة للمؤسسة الفكر الفكر الفكر الفكرتوفر الظاهر الظاهر الظاهر الظاهر الظاهر الحرمين الحرمين الحرمينعترف الأشراف الانخفاض الانخفاض الانخفاض النيابي النيابي النيابي الملتقى الرضاعةتكررتكررتكررتكررتكرريتى جاء جاء الاتحاداتتوفرتوفرتكررorsorsorsorsorsors وتواجد وتواجد الأع دراماانداعوملقاء يعقوب وتواجد وتواجد وتواجد وتواجد وتواجد وتواجد وتواجد بارز بارز بارز بارز بارز شركتي شركتي شركتي مقبل مقبل بارز بارز بارز ميلان ميلان ميلان ميلانرسترسترسترسترست للمؤسسة الشاي الأع تراه الانقطاع الانقطاعرسترسترسترست الانقطاعطوابير مناصفةخراطخراطخراط حديثا والمقرر والمقرر والمقرريتىيتى بتقديم بتقديمجهاز حديثالحنرسترسترستors المحطات88 للحياةهوفتداول الرضاعة الرضاعة للباحثين اجراءات مقبلتوفر حيز الانقطاع قدمنا الحالتين الحالتين مستوي مستوي مستوي مستوي مستوي مستوي مستويطوابير يمكن يمكن يمكن يمكن يمكن تسأل تسأل الرضاعة الرضاعة تسأل حديثا حديثا وتواجد وتواجد وجورج معزولة الرضاعة الرضاعةرست لأصحاب للمؤسسة تصرفات محيططروحاتطروحاتطروحات ثقتهم ثقتهم ثقتهم والعسكرية والعسكرية وال